In [2]:
###Spam Classifier in Tensorflow###
#Imports
import pandas as pd

In [5]:
# Import csv dataset into a pandas df
msgs = pd.read_csv("/content/SMSSpamCollection", sep="\t", names = ["label", "text"])
msgs

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [15]:
# Data preprocessing 
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


#initialize porter stemmer
stemmer = PorterStemmer()

text_corpus = []

for index in range(0, len(msgs)):
  text_reviews = re.sub('\W +','', msgs['text'][index]).replace('.', ' ').replace('  ', '')
  text_reviews = text_reviews.lower().split()
  text_reviews = [stemmer.stem(word) for word in text_reviews if word not in stopwords.words('english')]
  text_reviews = ' '.join(text_reviews)
  text_corpus.append(text_reviews)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# Convert the tokens into BOWs(Count Vectorizer)
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=3000)
X = vectorizer.fit_transform(text_corpus).toarray()
print(len(X))

5572


In [22]:
# extract the labels from the df
y = msgs['label'].map({'ham': 0, 'spam': 1}).to_numpy()
y

array([0, 0, 1, ..., 0, 0, 0])

In [24]:
# data split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [27]:
# Training
from sklearn.naive_bayes import MultinomialNB as nb

spam_model = nb().fit(X_train, y_train)

In [28]:
# Preds 
nb_preds = spam_model.predict(X_test)
# Metrics :) 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 

print('Accuracy Score \n',accuracy_score(nb_preds, y_test))
print('Confusion Matrix \n', confusion_matrix(nb_preds, y_test))
print('Classification Report \n', classification_report(nb_preds, y_test))

Accuracy Score 
 0.989237668161435
Confusion Matrix 
 [[963   9]
 [  3 140]]
Classification Report 
               precision    recall  f1-score   support

           0       1.00      0.99      0.99       972
           1       0.94      0.98      0.96       143

    accuracy                           0.99      1115
   macro avg       0.97      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115

